# Kaggle Competition: Patfinder Pawpularity score

## 7. Inference code

Import libraries

In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
import timm
import albumentations as A
import warnings
warnings.filterwarnings("ignore")

Device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


Load test data

Path names: images and model

In [3]:
path = '../input/petfinder-pawpularity-score/'
img_dir = path + 'test/'
data = 'test.csv'
path_model = '../models/backbone_effb0_bs32_lr00001_m09.pt'

Load in dataframe

In [4]:
test_data = pd.read_csv(path+data)
test_data['path'] = img_dir + test_data['Id'] + '.jpg'

print('Samples test:', len(test_data))
test_data.head()

Samples test: 8


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../input/petfinder-pawpularity-score/test/4128...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,../input/petfinder-pawpularity-score/test/43a2...
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,../input/petfinder-pawpularity-score/test/4e42...
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,../input/petfinder-pawpularity-score/test/80bc...
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,../input/petfinder-pawpularity-score/test/8f49...


Load in dataloader

In [5]:
img_size = 224
batch_size = 64
num_workers=os.cpu_count()
pin_memory=False

# To make transformations of the images
transform = A.Compose([
    A.Resize(img_size, img_size),
    A.Normalize()
])


class Dataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, label_dataframe, img_name, meta_x, transform=None):
        self.img_dir = img_dir
        self.img_label = label_dataframe
        self.transform = transform
        self.img_name = img_name
        self.meta_x = meta_x
        
    def __len__(self):
        return len(self.img_label)

    def __getitem__(self, ix):
        img_id = self.img_label[self.img_name][ix] 
        meta_x_ix = self.img_label[self.meta_x].iloc[ix]
        meta_x_ix = torch.tensor(meta_x_ix, dtype=torch.float32)

        img_path = self.img_dir + self.img_label[self.img_name][ix] + '.jpg'
        X_ix = plt.imread(img_path)
        if self.transform:
            X_ix = self.transform(image=X_ix)['image']
        X_ix = torch.tensor(X_ix).permute(2, 0, 1).float()

        return img_id, (X_ix, meta_x_ix)


meta_x_columns = [
    'Subject Focus','Eyes','Face','Near','Action','Accessory', 
    'Group', 'Collage','Human','Occlusion','Info','Blur'
    ]

dataset_test = Dataset(img_dir, test_data, 'Id', meta_x_columns, transform=transform)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory)


In [6]:
for i, (img_id, batch_X) in enumerate(dataloader_test):
    print(f"Batch #{i+1} of size -> image_data: {batch_X[0].shape} and meta_data: {batch_X[1].shape}")

Batch #1 of size -> image_data: torch.Size([8, 3, 224, 224]) and meta_data: torch.Size([8, 12])


Load model

In [7]:
class BackboneNet(nn.Module):
    def __init__(self, timm_model_name, len_meta_x, out_dim, pretrained=True, freezepretrained=False):
        super().__init__()

        self.efficientmodel = timm.create_model(timm_model_name, pretrained=pretrained)
        n_features = self.efficientmodel.classifier.in_features
        self.efficientmodel.classifier = nn.Identity()
        
        self.metamodel = nn.Sequential(
            nn.Linear(len_meta_x, 512), 
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3), 
            nn.Linear(512, 128), 
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        n_features += 128
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(n_features, out_dim)

        # freeze weights of efficientnet
        if freezepretrained:
            for param in self.efficientmodel.parameters():
                param.requires_grad = False


    def forward(self, image, meta_x):
        # backbone 1: efficient net
        x1 = self.efficientmodel(image)
        # backbone 2: meta model
        x2 = self.metamodel(meta_x)
        # concatenate backbones
        x = torch.cat((x1, x2), dim=1)
        # head
        x = self.dropout(x)
        x = self.fc(x)
        # drop dimension of size 1 (same as .view(-1)) (if not, broadcasting problems with y ground truth)
        x = x.squeeze()     
        return x

MODEL_NAME='tf_efficientnet_b0_ns'
model = BackboneNet(timm_model_name=MODEL_NAME, len_meta_x=12, out_dim=1, pretrained=False, freezepretrained=False)
model.to(device) 
model.load_state_dict(torch.load(path_model))
model.eval()

BackboneNet(
  (efficientmodel): EfficientNet(
    (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, t

In [8]:
tensordeprueba = torch.load('../others/tensordeprueba.pt', map_location=torch.device(device))
metadatadeprueba = torch.rand(5, 12).to(device)
y_hat = model(tensordeprueba, metadatadeprueba)
y_hat, y_hat.shape

(tensor([23.9125, 19.1448, 21.0734, 20.8529, 27.9514], device='cuda:0',
        grad_fn=<SqueezeBackward0>),
 torch.Size([5]))

Predictions

In [9]:
# make predictions on test set
id_img_all = []
predictions = []
with torch.no_grad():
    for i, (img_id , (batch_X, batch_x_meta)) in enumerate(dataloader_test):
        print(f"Batch #{i+1} of size -> image_data: {batch_X.shape} and meta_data: {batch_x_meta.shape}")
        y_hat = model(batch_X.to(device), batch_x_meta.to(device))
        predictions.append(y_hat)
        id_img_all.append(img_id)

# concat tensors/lists along batch dimension
predictions = torch.cat(predictions, dim=0)
id_img_all = [item for t in id_img_all for item in t]
#to numpy
predictions = predictions.detach().to("cpu").numpy()
predictions, predictions.shape

Batch #1 of size -> image_data: torch.Size([8, 3, 224, 224]) and meta_data: torch.Size([8, 12])


(array([29.073645, 25.349874, 24.205164, 25.99042 , 33.66132 , 26.157103,
        23.263758, 24.131908], dtype=float32),
 (8,))

Save prediction

In [10]:
# save predictions to csv named 'submission.csv'
submission = pd.DataFrame({'Id': id_img_all, 'Pawpularity': predictions})
submission.to_csv('submission.csv', index=False)
submission

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,29.073645
1,43a2262d7738e3d420d453815151079e,25.349874
2,4e429cead1848a298432a0acad014c9d,24.205164
3,80bc3ccafcc51b66303c2c263aa38486,25.990419
4,8f49844c382931444e68dffbe20228f4,33.661320
5,b03f7041962238a7c9d6537e22f9b017,26.157103
6,c978013571258ed6d4637f6e8cc9d6a3,23.263758
7,e0de453c1bffc20c22b072b34b54e50f,24.131908
